In [2]:
# import of libraries 
from sqlalchemy import create_engine, text
import pandas as pd
import plotly.express as px

In [3]:
# Call of our date in S3 
weather = pd.read_csv('s3://booking-scapping/city_weather_by_day.csv')
hotel = pd.read_csv('s3://booking-scapping/hotels_info.csv')

In [4]:
# Keep usefull data before sending it to our DB : take the weather by day of the top 5 city 

top_5_city_name = pd.read_csv('top_5_city_name.csv', names= ['city'])
top_5_city_list = [x.strip() for x in top_5_city_name['city']]

city_weather_top_5 = weather[weather['city'].isin(top_5_city_list)]
city_weather_top_5.head()

,Unnamed: 0,city,dt_object,main_weather,prepcipitation,temperature,latitude,longitude
0,0,Aigues Mortes,2023-02-15,Clouds,0.0,10.71000,43.565823,4.191284
1,1,Aigues Mortes,2023-02-16,Clouds,0.0,9.81125,43.565823,4.191284
2,2,Aigues Mortes,2023-02-17,Clear,0.0,10.83000,43.565823,4.191284
3,3,Aigues Mortes,2023-02-18,Clear,0.0,12.93875,43.565823,4.191284
4,4,Aigues Mortes,2023-02-19,Clear,0.0,12.17375,43.565823,4.191284


In [ ]:
# Clean the scrapping data before sending it to our DB

hotel[['lat', 'lon']] = hotel['latlng'].str.split(',', 1, expand=True)
hotel = hotel.drop(['depth','download_timeout','download_slot','latlng','download_latency'], axis=1)

hotel['Score'] = hotel['Score'].replace({ ',' : '.'}, regex=True)

hotel['Score'] = hotel['Score'].astype(float)
hotel['lat'] = hotel['lat'].astype(float)
hotel['lon'] = hotel['lon'].astype(float)

In [5]:
# send to our DB

engine = create_engine("sqlite:///:memory:", echo=True)
# engine = create_engine(f"mysql+pymysql://{DBUSER}:{DBPASS}@{DBHOST}:{PORT}/{DBNAME}", echo=True)
# engine = create_engine(f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}/{DBNAME}", echo=True)

city_weather_top_5.to_sql(
    "city_weather_top_5",
    engine
)

hotel.to_sql(
    "hotel",
    engine
)

2023-02-15 15:29:05,278 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("city_weather_top_5")
2023-02-15 15:29:05,279 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-15 15:29:05,279 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("city_weather_top_5")
2023-02-15 15:29:05,280 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-15 15:29:05,281 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-15 15:29:05,282 INFO sqlalchemy.engine.Engine 
CREATE TABLE city_weather_top_5 (
	"index" BIGINT, 
	"Unnamed: 0" BIGINT, 
	city TEXT, 
	dt_object TEXT, 
	main_weather TEXT, 
	prepcipitation FLOAT, 
	temperature FLOAT, 
	latitude FLOAT, 
	longitude FLOAT
)


2023-02-15 15:29:05,282 INFO sqlalchemy.engine.Engine [no key 0.00052s] ()
2023-02-15 15:29:05,283 INFO sqlalchemy.engine.Engine CREATE INDEX ix_city_weather_top_5_index ON city_weather_top_5 ("index")
2023-02-15 15:29:05,283 INFO sqlalchemy.engine.Engine [no key 0.00021s] ()
2023-02-15 15:29:05,284 INFO sqlalchemy.engine.Engine 

30

## Call tables from our BD and do visualization 

In [8]:
# Weather table 

stmt = text("SELECT * FROM city_weather_top_5 "
            )

weather = pd.read_sql(
        stmt,
        engine
    )

weather.head()

2023-02-15 15:29:05,423 INFO sqlalchemy.engine.Engine SELECT * FROM city_weather_top_5 
2023-02-15 15:29:05,423 INFO sqlalchemy.engine.Engine [generated in 0.00047s] ()


,index,Unnamed: 0,city,dt_object,main_weather,prepcipitation,temperature,latitude,longitude
0,0,0,Aigues Mortes,2023-02-15,Clouds,0.0,10.71000,43.565823,4.191284
1,1,1,Aigues Mortes,2023-02-16,Clouds,0.0,9.81125,43.565823,4.191284
2,2,2,Aigues Mortes,2023-02-17,Clear,0.0,10.83000,43.565823,4.191284
3,3,3,Aigues Mortes,2023-02-18,Clear,0.0,12.93875,43.565823,4.191284
4,4,4,Aigues Mortes,2023-02-19,Clear,0.0,12.17375,43.565823,4.191284


In [9]:
# Add a proportinal colum on temperature for better see the variation 
city_weather_top_5['temp_cube'] = city_weather_top_5['temperature']**3
city_weather_top_5.head()

city_weather_top_5['dt_object'] = city_weather_top_5['dt_object'].astype(str)

In [13]:
fig = px.scatter_mapbox(city_weather_top_5, lat="latitude", lon="longitude",
                        color="temperature", size ='temp_cube', zoom=7, mapbox_style="carto-positron",
                        animation_frame = "dt_object", animation_group = 'temperature',
                        title="Top 5 best weather cities")
fig.show()

In [11]:
# Hotel info
stmt = text("SELECT * FROM hotel "
            )

df = pd.read_sql(
        stmt,
        engine
    )

df.head()

2023-02-15 15:29:05,874 INFO sqlalchemy.engine.Engine SELECT * FROM hotel 
2023-02-15 15:29:05,875 INFO sqlalchemy.engine.Engine [generated in 0.00075s] ()


,index,Unnamed: 0,hotel name,Url to its booking.com page,Score,Text description,lat,lon
0,0,0,Aux MARCHES DU PALAIS - SPACIEUX - TERRASSE - ...,https://www.booking.com/hotel/fr/nouveau-aux-m...,9.2,None,43.952055,4.805463
1,1,1,KAROUBA.31,https://www.booking.com/hotel/fr/karouba-31.fr...,9.5,None,43.947607,4.807996
2,2,2,Maison XIXe et Jardin en Intramuros,https://www.booking.com/hotel/fr/maison-xixe-e...,9.0,None,43.950344,4.811538
3,3,3,Hôtel Cloitre Saint Louis Avignon,https://www.booking.com/hotel/fr/clarioncloitr...,7.6,None,43.943808,4.804947
4,4,4,Cute and cozy town-house of 130m2 in Avignon,https://www.booking.com/hotel/fr/cute-and-cozy...,8.4,None,43.946141,4.813088


In [12]:
fig = px.scatter_mapbox(hotel[hotel['Score'].notna()], lat="lat", lon="lon", 
                        zoom=7, size="Score", color="Score", mapbox_style="carto-positron", 
                        title = 'Top 20 hotels in our top 5 cities')
fig.show()